In [ ]:
pip install yolov5

In [ ]:
pip install flask flask-restful torch torchvision opencv-python numpy ultralytics deep_sort_realtime pytube yt_dlp

In [ ]:
from flask import Flask, request
from flask_restful import Resource, Api
import torch
import cv2
from torchvision import transforms as T
from yolov5 import detect  # Assuming detect function is defined in yolov5.py for inference

app = Flask(__name__)
api = Api(app)

# Function to process video frames
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return "Error: Unable to open video file"

    # Initialize YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).fuse().eval()

    results = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection using YOLOv5
        # Assuming detect function takes frame and returns detections
        detections = detect(model, frame)

        # Draw bounding boxes on frame
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{cls}: {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Convert frame to JPEG format for displaying or streaming
        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        results.append(frame_bytes)

    cap.release()
    return results

# Flask Restful API resource
class ObjectDetection(Resource):
    def post(self):
        video_file = request.files['video']
        video_path = '/tmp/input_video.mp4'  # Save uploaded video to a temporary location
        video_file.save(video_path)

        # Process video frames
        frames = process_video(video_path)

        return frames

api.add_resource(ObjectDetection, '/detect')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


app.run(port=8000)
app.run(host='0.0.0.0',port=5000)